In [19]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [20]:
df1 = pd.read_csv("v1-start.csv")

# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [21]:
df1.head()

,type,prefix,name,winery,size,vintage,sales_price_eur,qty
0,SPARKLING,NaN,Praeclarus brut,Kelleri St. Pauls,NaN,NaN,39.0,9
1,SPARKLING,NaN,Haderburg Pas Dosè Millesimato,Sektkellerei Haderburg,NaN,2024.0,57.0,6
2,SPARKLING,NaN,Athesis Brut Rosè,Sektkellerei Kettmeier,NaN,2020.0,46.0,2
3,SPARKLING,NaN,Ferrari Maximum Blanc de Blanc Brut,Ferrari,NaN,NaN,52.0,3
4,SPARKLING,NaN,Corvezzo Rosè Family Collection Millesimato,Corvezzo,NaN,NaN,24.0,5


In [22]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [23]:
import re

# copy values from original columns to new columns
df['external_id'] = None
df['type'] = df1['type']
df['name'] = df1['name']
df['winery_name'] = df1['winery']
# First extract vintage from name
df["vintage"] = df1["vintage"]
df["quantity"] = df1["qty"]
df["storage_area"] = None
df["internal_notes"] = None
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,SPARKLING,Praeclarus brut,Kelleri St. Pauls,NaN,NaN,NaN,NaN,NaN,9,None,None,True
1,NaN,SPARKLING,Haderburg Pas Dosè Millesimato,Sektkellerei Haderburg,NaN,NaN,2024.0,NaN,NaN,6,None,None,True
2,NaN,SPARKLING,Athesis Brut Rosè,Sektkellerei Kettmeier,NaN,NaN,2020.0,NaN,NaN,2,None,None,True
3,NaN,SPARKLING,Ferrari Maximum Blanc de Blanc Brut,Ferrari,NaN,NaN,NaN,NaN,NaN,3,None,None,True
4,NaN,SPARKLING,Corvezzo Rosè Family Collection Millesimato,Corvezzo,NaN,NaN,NaN,NaN,NaN,5,None,None,True


In [24]:
# size
df['size'] = df1["size"].fillna("BOTTLE")

# fill empty prices with 0
df1['price_eur'] = df1['sales_price_eur'].fillna(0)

# convert prices to cents
df['price'] = df1['price_eur'].apply(lambda x: int(float(str(x).replace('€', ''))*100)) # convert to cents
df['purchase_price'] = 0

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,NaN,SPARKLING,Praeclarus brut,Kelleri St. Pauls,NaN,BOTTLE,NaN,3900,0,9,None,None,True
1,NaN,SPARKLING,Haderburg Pas Dosè Millesimato,Sektkellerei Haderburg,NaN,BOTTLE,2024.0,5700,0,6,None,None,True
2,NaN,SPARKLING,Athesis Brut Rosè,Sektkellerei Kettmeier,NaN,BOTTLE,2020.0,4600,0,2,None,None,True
3,NaN,SPARKLING,Ferrari Maximum Blanc de Blanc Brut,Ferrari,NaN,BOTTLE,NaN,5200,0,3,None,None,True
4,NaN,SPARKLING,Corvezzo Rosè Family Collection Millesimato,Corvezzo,NaN,BOTTLE,NaN,2400,0,5,None,None,True


In [25]:
# discard rows with 0 qty
df = df[df["quantity"] > 0]

df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,,SPARKLING,Praeclarus brut,Kelleri St. Pauls,,BOTTLE,0,3900,0,9,,,True
1,,SPARKLING,Haderburg Pas Dosè Millesimato,Sektkellerei Haderburg,,BOTTLE,2024,5700,0,6,,,True
2,,SPARKLING,Athesis Brut Rosè,Sektkellerei Kettmeier,,BOTTLE,2020,4600,0,2,,,True
3,,SPARKLING,Ferrari Maximum Blanc de Blanc Brut,Ferrari,,BOTTLE,0,5200,0,3,,,True
4,,SPARKLING,Corvezzo Rosè Family Collection Millesimato,Corvezzo,,BOTTLE,0,2400,0,5,,,True


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [26]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False